In [182]:
import cameo
import cobra
import os
import pandas as pd
from cobra.core import Metabolite, Reaction

In [190]:
model = cameo.load_model(dirmodels+'iML1515_PKT.json')

In [191]:
model.metabolites.acac_c

Metabolite identifier,acac_c
Name,Acetoacetate
Memory address,0x07fc8bb216940
Formula,C4H5O3
Compartment,c
In 2 reaction(s),"ACACt2pp, ACACCT"


In [193]:
model2 = cameo.load_model(dirmodels+'iML1515.json')
model2

Name,iML1515
Memory address,0x07fc8b2162e10
Number of metabolites,1877
Number of reactions,2712
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


In [195]:
model2.reactions.ACACCT

Reaction identifier,ACACCT
Name,Acetyl-CoA:acetoacetyl-CoA transferase
Memory address,0x07fc8d1f92b38
Stoichiometry,acac_c + accoa_c --> aacoa_c + ac_c Acetoacetate + Acetyl-CoA --> Acetoacetyl-CoA + Acetate
GPR,(b2222 and b2221) or b1694
Lower bound,0.0
Upper bound,1000.0


In [176]:
reacts_del = ['ACACDCBX', 'IPRNADH', 'IPRNADPH', 'EX_acetone_e', 'EX_2ppoh_e']
mets_del = ['acetone_c', '2ppoh_c']

In [177]:
model.remove_reactions(reacts_del)

In [178]:
model.remove_metabolites([model.metabolites.get_by_id(m_id) for m_id in mets_del])

In [179]:
cobra.io.save_json_model(model,dirmodels+'iML1515_PKT_new.json')

In [201]:
dirmodels = '/media/frek/Storage/projects/chemocobra/static/metabolic/models_old/'
dirout = '/media/frek/Storage/projects/chemocobra/static/metabolic/models/'

In [79]:
os.listdir(dirmodels)

['iML1515_PKT.json',
 'iML1515.json',
 'iJO1366.json',
 'RECON1.json',
 'iHN637.json']

In [183]:
# os.system('cp /home/frek/Desktop/ibn_pathway.xlsx /media/frek/Storage/projects/chemocobra/ipynb')
data_ibn = pd.read_excel('ibn_pathway.xlsx',index_col=0,sheet_name=None)

In [184]:
data_ibn['reactions']

,name,lower_bound,upper_bound,gene_reaction_rule,reaction
id,,,,,
HMGCS,HMG-CoA Synthase,-1000,1000,S318,aacoa_c + accoa_c + h2o_c <==> hmgcoa_c + coa_...
ECH,HMG-CoA Dehydratase,-1000,1000,ECH,hmgcoa_c <==> mgcoa_c + h2o_c
AIBAB,MG-CoA Decarboxylase,0,1000,(aibA and aibB),mgcoa_c + h_c → mccoa_c + co2_c
TESB,Acyl-CoA Thioesterase,-1000,1000,b0452,mccoa_c + h2o_c → mca_c + coa_c
SS5,Prenate decarboxylase,0,1000,ss5,mca_c → ibn_c + co2_c
EX_IBN_e,IBN Exchange,0,1000,NaN,ibn_c →
ACACDCBX,Acetoacetate Decarboxylase,0,1000,NaN,acac_c + h_c --> acetone_c + co2_c
IPRNADH,Propane 2-Monooxygenase (EC 1.14.13.227),-1000,1000,NaN,acetone_c + h_c + nadh_c --> ipa_c + nad_c
IPRNADPH,Isopropanol Dehydrogenase (EC 1.1.1.80),-1000,1000,NaN,acetone_c + h_c + nadph_c --> ipa_c + nadp_c


In [204]:
mets = {}
## Charge metabolites
for m_id in data_ibn['metabolites'].index:
    mets[m_id] = Metabolite(
        id = m_id,
        name = data_ibn['metabolites'].loc[m_id,'name'],
        formula = data_ibn['metabolites'].loc[m_id,'formula'],
        compartment = data_ibn['metabolites'].loc[m_id,'compartment']
    )
#     print(m_id,'charge:',mets[m_id].charge)
    
for filename in [f for f in os.listdir(dirmodels) if 'old' not in f]:
    model = cameo.load_model(dirmodels+filename)
    model.add_metabolites(mets.values())    
    ## add reactions
    for r_id in data_ibn['reactions'].index:
        r = Reaction(
            id = r_id,
            name = data_ibn['reactions'].loc[r_id,'name'],
            subsystem = 'IBN Pathway D',
            lower_bound = data_ibn['reactions'].loc[r_id,'lower_bound'],
            upper_bound = data_ibn['reactions'].loc[r_id,'upper_bound']
        )
        model.add_reaction(r)
        r.reaction = data_ibn['reactions'].loc[r_id,'reaction'].replace('→','-->')
        balanced = r.check_mass_balance()
    
    try:
        ACACCT = model.reactions.get_by_id('ACACCT')
    except:
        pass
    else:
        ACACCT.lower_bound = -1000
        print('ACACCT changed for',model.id)
            
    if 'iML1515' in model.id:
        model.objective = 'EX_acetone'
        res = model.optimize()
        print(model.id, res.objective_value)
    
    if model.id == 'iML1515_PKT_old':
        model.id = 'iML1515_PKT'
    cobra.io.save_json_model(model, dirout+filename)
    
    print(model.id, 'OK')
    

ACACCT changed for iML1515_PKT_old
iML1515_PKT_old 14.792380952380945
iML1515_PKT OK
ACACCT changed for iML1515
iML1515 13.841159420289856
iML1515 OK
ACACCT changed for iJO1366
iJO1366 OK
RECON1 OK
unknown metabolite 'acac_c' created
iHN637 OK


In [198]:
r = model.reactions.get_by_id('ECH')
r

Reaction identifier,ECH
Name,HMG-CoA Dehydratase
Memory address,0x07fc8aa6012e8
Stoichiometry,hmgcoa_c <=> h2o_c + mgcoa_c Hydroxymethylglutaryl CoA <=> H2O H2O + 3-Methylglutaconyl-CoA
GPR,
Lower bound,-1000
Upper bound,1000


In [199]:
r = model.reactions.get_by_id('HMGCS')
for m in r.metabolites:
    print(m.id, r.metabolites[m], m.elements)

aacoa_c -1 {'C': 25, 'H': 36, 'N': 7, 'O': 18, 'P': 3, 'S': 1}
accoa_c -1 {'C': 23, 'H': 34, 'N': 7, 'O': 17, 'P': 3, 'S': 1}
h2o_c -1 {'H': 2, 'O': 1}
hmgcoa_c 1 {'C': 27, 'H': 39, 'N': 7, 'O': 20, 'P': 3, 'S': 1}
coa_c 1 {'C': 21, 'H': 32, 'N': 7, 'O': 16, 'P': 3, 'S': 1}
h_c 1 {'H': 1}


In [200]:
model

Name,iHN637
Memory address,0x07fc8aa7be668
Number of metabolites,706
Number of reactions,796
Number of groups,0
Objective expression,1.0*BIOMASS_Cl_DSM_WT_46p666M1 - 1.0*BIOMASS_Cl_DSM_WT_46p666M1_reverse_a44fc
Compartments,"cytosol, extracellular space"


In [45]:
r.check_mass_balance()

{}